# 0) Colab environment setup and data preparation

This notebook helps you:
- Check Google Drive mount
- Install dependencies and initialize DuckDB (via repository script)
- Download interest rate data for IV calculation

In [1]:
# 1) Check Google Drive mount (no automatic mount)
import os

# Detect if running inside Google Colab
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

DRIVE_ROOT = "/content/drive"
MYDRIVE = os.path.join(DRIVE_ROOT, "MyDrive")

if IN_COLAB:
    root_exists = os.path.exists(DRIVE_ROOT)
    mydrive_exists = os.path.exists(MYDRIVE)
    is_mounted = (os.path.ismount(DRIVE_ROOT) or mydrive_exists)

    print("Environment: Colab")
    print(f"Drive root exists: {root_exists}")
    print(f"MyDrive exists: {mydrive_exists}")
    print(f"Mounted: {is_mounted}")

    if not is_mounted:
        print("\n[Hint] Drive seems not mounted. If you need Drive, run:")
        print("from google.colab import drive")
        print("drive.mount('/content/drive')")
else:
    print("Environment: not Colab. Skipping Google Drive mount check.")


Environment: Colab
Drive root exists: True
MyDrive exists: True
Mounted: True


In [2]:
# 2) Set repository path and validate
import os
REPO_DIR = '/content/drive/MyDrive/00_EUREX/eurex-liquidity-demo'
assert os.path.exists(REPO_DIR), f'Repo not found: {REPO_DIR}'
REPO_DIR

'/content/drive/MyDrive/00_EUREX/eurex-liquidity-demo'

In [3]:
# 3) Install dependencies and initialize DuckDB (script)
!bash "$REPO_DIR/warehouse/setup_colab_duckdb.sh" "$REPO_DIR"

bash: /content/drive/MyDrive/00_EUREX/eurex-liquidity-demo/warehouse/setup_colab_duckdb.sh: No such file or directory


In [4]:
# 4) Verify DuckDB connection
import duckdb, os
db_path = os.path.join(REPO_DIR, 'warehouse', 'eurex.duckdb')
con = duckdb.connect(db_path)
print('DuckDB version:', duckdb.__version__)
print(con.sql("SELECT 1 AS ok").df())
con.close()


DuckDB version: 1.3.2
   ok
0   1


# 5) Extract original dataset

Extract the original Eurex sample archive `Sample_Eurex_20201201_10MktSegID.tar.gz`
from Google Drive into project folder `eurex-liquidity-demo/data_raw` or Colab local (`/content`)

Instructions:
- Set `TAR_PATH` to your `.tar.gz` on Drive.
- Set `DEST_DIR` to your project folder `data_raw` or Colab local (`/content`).
- Adjust `--progress-every`, `--list-top`, and `--show-tree` as needed.

In the current demo, we only extract segments 48 and 50
- 48: FSTK-ADSG (Futures on Adidas AG)
- 50: OSTK-ADS (Options on Adidas AG)


In [13]:
# # 5.1) Inspect tar.gz file structure (without extraction)
# import tarfile

# TAR_PATH = "/content/drive/MyDrive/00_EUREX/sample_data/Sample_Eurex_20201201_10MktSegID.tar.gz"

# with tarfile.open(TAR_PATH, 'r:gz') as tar:
#     names = tar.getnames()
#     print(f"Total entries: {len(names)}\n")
#     for name in names:
#         print(name)

Total entries: 81

Sample_Eurex_20201201_10MktSegID
Sample_Eurex_20201201_10MktSegID/821
Sample_Eurex_20201201_10MktSegID/821/ISC_821_20201201.csv
Sample_Eurex_20201201_10MktSegID/821/IS_821_20201201.csv
Sample_Eurex_20201201_10MktSegID/821/MISC_821_20201201.csv
Sample_Eurex_20201201_10MktSegID/821/DS_821_20201201.csv
Sample_Eurex_20201201_10MktSegID/821/DI_821_20201201.csv
Sample_Eurex_20201201_10MktSegID/821/PSC_821_20201201.csv
Sample_Eurex_20201201_10MktSegID/589
Sample_Eurex_20201201_10MktSegID/589/PSC_589_20201201.csv
Sample_Eurex_20201201_10MktSegID/589/DS_589_20201201.csv
Sample_Eurex_20201201_10MktSegID/589/MISC_589_20201201.csv
Sample_Eurex_20201201_10MktSegID/589/DI_589_20201201.csv
Sample_Eurex_20201201_10MktSegID/589/IS_589_20201201.csv
Sample_Eurex_20201201_10MktSegID/1374
Sample_Eurex_20201201_10MktSegID/1374/DS_1374_20201201.csv
Sample_Eurex_20201201_10MktSegID/1374/DI_1374_20201201.csv
Sample_Eurex_20201201_10MktSegID/1374/IS_1374_20201201.csv
Sample_Eurex_20201201_10M

In [ ]:
# '''
# Uncomment the code below to extract the dataset to Colab local.
# '''

# # 6) Extract dataset to Colab local
# # Set TAR_PATH and DEST_DIR, then run the command below.
# # Example uses the file path you referenced earlier. Adjust if needed.
# TAR_PATH = "/content/drive/MyDrive/00_EUREX/sample_data/Sample_Eurex_20201201_10MktSegID.tar.gz"
# DEST_DIR = f"content/data_raw"

# !python "{REPO_DIR}/scripts/extract_segments_to_drive.py" \
#   --tar "{TAR_PATH}" \
#   --dest "{DEST_DIR}" \
#   --segments 48 50 \
#   --progress-every 10 \
#   --list-top 20 \
#   --show-tree

Destination (Drive) usage:
Total: 225.83 GB  |  Used: 42.18 GB  |  Free: 183.64 GB

Tar size: 2.57 GB

[INFO] Extracting segments ['48', '50']
[INFO] From: /content/drive/MyDrive/00_EUREX/sample_data/Sample_Eurex_20201201_10MktSegID.tar.gz
[INFO] To  : content/data_raw
Progress: 81/81 (100.0%)  |  Extracted: 15  |  Elapsed: 261.7s  
[OK] Extraction completed
Elapsed: 261.7 s  |  Members scanned: 81  |  Extracted: 15

Extracted files (under dest): 15  |  Total size: 1.28 GB

Top 20 files by size:
  1. Sample_Eurex_20201201_10MktSegID/50/DI_50_20201201.csv                    1.08 GB
  2. Sample_Eurex_20201201_10MktSegID/50/DS_50_20201201.csv                  204.71 MB
  3. Sample_Eurex_20201201_10MktSegID/48/DI_48_20201201.csv                    1.22 MB
  4. Sample_Eurex_20201201_10MktSegID/48/DS_48_20201201.csv                  990.78 KB
  5. Sample_Eurex_20201201_10MktSegID/50/IS_50_20201201.csv                  146.84 KB
  6. Sample_Eurex_20201201_10MktSegID/48/IS_48_20201201.csv     

## Extract FULL dataset to Colab local SSD

**Important storage strategy:**
- **Raw data**: Extract to Colab local SSD (`/content/`) - temporary, fast, free space
- **Intermediate products**: Save to Google Drive (slices, aggregates, Parquet files) - persistent

This extraction includes ALL segments for full-day processing.

In [ ]:
# Extract FULL dataset to Colab local SSD (all segments)
TAR_PATH = "/content/drive/MyDrive/00_EUREX/sample_data/Sample_Eurex_20201201_10MktSegID.tar.gz"
DEST_LOCAL = "/content"  # Colab local SSD - fast and free

!python "{REPO_DIR}/scripts/extract_to_colab_local.py" \
  --tar "{TAR_PATH}" \
  --dest "{DEST_LOCAL}"

# Expected output: /content/Sample_Eurex_20201201/
RAW_LOCAL = "/content/Sample_Eurex_20201201"
print(f"\n✅ Raw data extracted to: {RAW_LOCAL}")

Disk usage on Colab local (/content/):
Total: 225.83 GB  |  Used: 38.28 GB  |  Free: 187.54 GB

Tar size: 2.57 GB  |  Estimated need: 10.29 GB

[INFO] Extracting: /content/drive/MyDrive/00_EUREX/sample_data/Sample_Eurex_20201201_10MktSegID.tar.gz
[INFO] Destination: /content
Members: 81
Progress: 50/81 (61.7%)  |  Elapsed: 154.9s  |  Sample_Eurex_20201201_10MktSegID/1176/MISC_1176_20201201.csv

## Create symlink to raw data folder

Create a shortcut in your Drive repo pointing to the Colab local raw data.
This makes it easy to navigate the raw data structure from the Drive repo folder.

**📖 See `data_raw/COLAB_RAW_DATA_STRUCTURE.md` for full structure documentation**

### Alternative: Detailed depth analysis (slower but more accurate)

Use the `check_max_depth.py` script for precise depth detection.
This requires first inferring the schema for each segment.

In [17]:
# Inspect maximum depth for all segments (PROPER PARSING)
import os
import sys
import glob
import json
import pandas as pd
from pathlib import Path
from collections import Counter

# Add src to path
sys.path.append(str(Path(REPO_DIR) / "src"))

from eurex_liquidity.parser import extract_entry_tokens_from_di_line, infer_di_mapping, tokens_to_event

RAW_LOCAL = "/content/Sample_Eurex_20201201_10MktSegID"

# Find all segments
segment_dirs = [d for d in Path(RAW_LOCAL).iterdir() if d.is_dir() and d.name.isdigit()]
segments = sorted([int(d.name) for d in segment_dirs])

print(f"Found {len(segments)} segments: {segments}")
print("="*70)

results = []

for seg in segments:
    seg_dir = Path(RAW_LOCAL) / str(seg)
    
    # Find DI file
    di_files = list(seg_dir.glob("DI_*.csv"))
    if not di_files:
        print(f"Segment {seg:5d}: ⚠️  No DI file found")
        continue
    
    di_file = di_files[0]
    file_size_mb = di_file.stat().st_size / 1024 / 1024
    
    try:
        # Step 1: Infer schema from DI file
        print(f"{seg:5d}:", end="", flush=True)
        
        # Read first 200 lines for schema inference
        sample_lines = []
        with open(di_file, 'r', encoding='utf-8', errors='ignore') as f:
            for i, line in enumerate(f):
                if i >= 200:
                    break
                sample_lines.append(line.strip())
        
        mapping = infer_di_mapping(sample_lines, sample_limit=200)
        
        if mapping is None:
            print(f" ❌ Schema inference failed")
            continue
        
        # Step 2: Parse entries and collect price levels
        level_counts = Counter()
        lines_scanned = 0
        sample_limit = 5000
        
        with open(di_file, 'r', encoding='utf-8', errors='ignore') as f:
            for i, line in enumerate(f):
                if i >= sample_limit:
                    break
                lines_scanned += 1
                
                entries = extract_entry_tokens_from_di_line(line.strip())
                for entry_tokens in entries:
                    evt = tokens_to_event(entry_tokens, mapping)
                    level = evt.get('price_level')
                    
                    if level is not None and isinstance(level, int) and 0 <= level <= 100:
                        level_counts[level] += 1
        
        max_level = max(level_counts.keys()) if level_counts else 0
        
        # Calculate distribution
        l1_pct = (level_counts.get(0, 0) / sum(level_counts.values()) * 100) if level_counts else 0
        
        results.append({
            'Segment': seg,
            'Max_Level': max_level,
            'Suggested_L': min(max_level + 1, 20),
            'L1_Entries_%': round(l1_pct, 1),
            'DI_Size_MB': round(file_size_mb, 2),
            'Lines_Scanned': lines_scanned,
            'Total_Entries': sum(level_counts.values())
        })
        
        print(f" Max level: {max_level:2d} (L1: {l1_pct:.1f}%)")
        
    except Exception as e:
        print(f" ❌ Error: {e}")
        import traceback
        traceback.print_exc()

print("="*70)

# Create summary DataFrame
if results:
    df_depth = pd.DataFrame(results).sort_values('Segment')
    print("\n📊 DEPTH SUMMARY TABLE")
    print("="*70)
    display(df_depth)
    
    # Distribution stats
    print("\n📈 DEPTH DISTRIBUTION:")
    print("-"*70)
    for _, row in df_depth.iterrows():
        seg = int(row['Segment'])
        max_lvl = int(row['Max_Level'])
        if max_lvl <= 5:
            cat = "🟢 Low depth (suitable for L5)"
        elif max_lvl <= 10:
            cat = "🟡 Medium depth (use L10)"
        elif max_lvl <= 15:
            cat = "🟠 High depth (use L15)"
        else:
            cat = "🔴 Very deep (use L20)"
        print(f"  Segment {seg:5d}: L{max_lvl:2d}  {cat}")
    
    # Save summary to Drive
    summary_path = f"{REPO_DIR}/data_raw/segment_depth_summary.json"
    summary_data = {
        'analysis_date': pd.Timestamp.now().isoformat(),
        'segments_analyzed': len(results),
        'results': results  # Use raw results list instead of DataFrame
    }
    with open(summary_path, 'w') as f:
        json.dump(summary_data, f, indent=2)
    print(f"\nSummary saved to: {summary_path}")
    
else:
    print("⚠️  No segments processed successfully")

Found 10 segments: [48, 50, 589, 688, 702, 821, 1176, 1209, 1373, 1374]
   48: Max level:  5 (L1: 35.0%)
   50: Max level:  5 (L1: 25.2%)
  589: Max level:  5 (L1: 42.6%)
  688: Max level:  2 (L1: 37.4%)
  702: Max level:  5 (L1: 73.3%)
  821: Max level:  5 (L1: 63.0%)
 1176: Max level:  5 (L1: 19.2%)
 1209: Max level:  4 (L1: 34.3%)
 1373: Max level:  5 (L1: 10.7%)
 1374: Max level:  5 (L1: 11.8%)

📊 DEPTH SUMMARY TABLE


,Segment,Max_Level,Suggested_L,L1_Entries_%,DI_Size_MB,Lines_Scanned,Total_Entries
0,48,5,6,35.0,1.22,5000,13415
1,50,5,6,25.2,1103.96,5000,153570
2,589,5,6,42.6,221.08,5000,8809
3,688,2,3,37.4,366.51,5000,9453
4,702,5,6,73.3,0.46,524,7281
5,821,5,6,63.0,54.77,5000,19668
6,1176,5,6,19.2,9610.09,5000,149313
7,1209,4,5,34.3,0.00,4,35
8,1373,5,6,10.7,921.57,5000,39015
9,1374,5,6,11.8,0.38,682,6873



📈 DEPTH DISTRIBUTION:
----------------------------------------------------------------------
  Segment    48: L 5  🟢 Low depth (suitable for L5)
  Segment    50: L 5  🟢 Low depth (suitable for L5)
  Segment   589: L 5  🟢 Low depth (suitable for L5)
  Segment   688: L 2  🟢 Low depth (suitable for L5)
  Segment   702: L 5  🟢 Low depth (suitable for L5)
  Segment   821: L 5  🟢 Low depth (suitable for L5)
  Segment  1176: L 5  🟢 Low depth (suitable for L5)
  Segment  1209: L 4  🟢 Low depth (suitable for L5)
  Segment  1373: L 5  🟢 Low depth (suitable for L5)
  Segment  1374: L 5  🟢 Low depth (suitable for L5)

Summary saved to: /content/drive/MyDrive/00_EUREX/eurex-liquidity-demo/data_raw/segment_depth_summary.json
